In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10

X = tf.placeholder(tf.float32, [None, n_input])
# 어라? Y? Y는 해당 숫자를 힌트로 넣어주는 용도로 쓸 것
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

W0718 11:36:46.130363 10944 deprecation.py:323] From <ipython-input-1-8d6b7429cfd3>:6: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0718 11:36:46.130363 10944 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0718 11:36:46.130363 10944 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement 

Extracting ./mnist/data/train-images-idx3-ubyte.gz


W0718 11:36:46.356442 10944 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0718 11:36:46.358438 10944 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0718 11:36:46.401290 10944 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please 

Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
# 신경망 구성 tf.layers를 써보자, 변수 선언필요 없다. tf.variable_scope로 스코프 지정가능
def generator(noise, labels):
    with tf.variable_scope('generator'):
        inputs = tf.concat([noise, labels], 1)
        # 은닉층 제작
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        # 출력층 제작
        output=tf.layers.dense(hidden, n_input, activation=tf.nn.sigmoid)
    return output

In [3]:
# 생설자와 마찬가지로 구분자 신경망 제작, 구분자는 진짜, 가짜 판별때 똑같은 변수 사용
def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        if reuse:
            scope.reuse_variables()
        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        # 출력값에 활성화 안쓰는건 손실값 계산에 sigmoid_cross_entropy_with_logits함수를 쓰기 위함
        output = tf.layers.dense(hidden, 1, activation=None)
    return output

In [4]:
# 노이즈 균등불포 생성
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size=[batch_size, n_noise])

In [5]:
# 생성자 구성, 진짜 이미지 데이터와 생성자가 만든 이미지 데이터 이용 구분자 제작
# 생성자에 레이블정보 추가로 추후 레이블 정보에 해당하는 이미지 생성 하도록 유도
# 이미지 구분자에서 ㅅ용한 변수들 재사용하도록 reuse옵션 True
G = generator(Z, Y)
D_real = discriminator(X, Y)
D_gene = discriminator(G, Y, True)

W0718 11:36:48.472099 10944 deprecation.py:323] From <ipython-input-2-15380f1475ff>:6: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0718 11:36:48.521017 10944 deprecation.py:506] From c:\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [19]:
# D_real은 1에 가깝게 D_gene은 0에 가깝게, sigmoid_cross_entropy_with_ligits함수 이용하면 코드 간편하게 작성가능
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.zeros_like(D_gene)))
# 둘을 합친 것 이 값을 최소화 하면 구분자 학습
# D_real은 1에 가까워야 되서 결과값만큼 1로 채운 것 loss_D_real에
# D_gene은 0에 가까워야 되서 결과값만큼 0으로 채워 것 loss_D_gene에
# 넣은 뒤 비교
loss_D = loss_D_real + loss_D_gene

In [20]:
# loss_G구하기, 생성자 학습 손실값, sigmoid_cross_entropy_with_logits함수 이용 D_gene이 1에 가깝게
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

In [21]:
# tf.get_collection함수 이용, 각 스코프에서 사용된 변수 가져온 뒤 최적화에 사용할 손실함수화 넣어 학습 모델 구성
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')
train_D = tf.train.AdamOptimizer().minimize(loss_D, var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G, var_list=vars_G)

In [22]:
# 학습코드, 플레이스 홀더 Y입력값이 batch_ys인 것에 주의
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Y: batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Y: batch_ys, Z: noise})
    
    print('Epoch:', '%04d'%epoch, 'D loss: {:.4}'.format(loss_val_D), 'G loss: {:.4}'.format(loss_val_G))
    
    # 학습 중간 중간에 생성자로 만든 이미지를 저장하는 코드를 작성
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Y: mnist.test.labels[:sample_size], Z: noise})
        
        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size,2))
        
        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()
            
            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28,28)))
        plt.savefig('samples2/{}_2.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
print('최적화 완료!')

Epoch: 0000 D loss: 0.004842 G loss: 7.502
Epoch: 0001 D loss: 0.01061 G loss: 7.059
Epoch: 0002 D loss: 0.0004952 G loss: 10.56
Epoch: 0003 D loss: 0.001814 G loss: 10.45
Epoch: 0004 D loss: 0.0004315 G loss: 12.01
Epoch: 0005 D loss: 7.08e-05 G loss: 11.17
Epoch: 0006 D loss: 0.0002703 G loss: 9.362
Epoch: 0007 D loss: 1.476e-05 G loss: 12.48
Epoch: 0008 D loss: 2.012e-05 G loss: 12.47
Epoch: 0009 D loss: 8.364e-05 G loss: 11.59
Epoch: 0010 D loss: 3.559e-06 G loss: 13.03
Epoch: 0011 D loss: 2.802e-06 G loss: 13.34
Epoch: 0012 D loss: 6.112e-05 G loss: 11.97
Epoch: 0013 D loss: 6.243e-06 G loss: 13.02
Epoch: 0014 D loss: 8.291e-06 G loss: 17.26
Epoch: 0015 D loss: 1.636e-06 G loss: 14.99
Epoch: 0016 D loss: 5.352e-06 G loss: 17.93
Epoch: 0017 D loss: 2.607e-06 G loss: 14.53
Epoch: 0018 D loss: 1.144e-05 G loss: 12.68
Epoch: 0019 D loss: 2.768e-06 G loss: 13.65
Epoch: 0020 D loss: 8.53e-10 G loss: 21.66
Epoch: 0021 D loss: 7.287e-07 G loss: 14.7
Epoch: 0022 D loss: 4.198e-06 G loss: 1